<a href="https://colab.research.google.com/github/A-Burnhard/Seoul-Bike-Sharing-Demand-Analysis/blob/main/Bikes_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
import copy
import seaborn as sns
import tensorflow as tf
from sklearn.linear_model import LinearRegression

**Dataset**

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Source:

Data Source :http://data.seoul.go.kr/
SOUTH KOREA PUBLIC HOLIDAYS. URL: publicholidays.go.kr

In [3]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive//Machine Learning/SeoulBikeData.csv'


Mounted at /content/drive


In [9]:
dataset_cols =   dataset_cols = ["bike_count", "hour", "temp", "humidity", "wind", "visibility", "dew_pt_temp", "radiation", "rain", "snow", "functional"]
df = pd.read_csv(file_path,encoding='ISO-8859-1',names=dataset_cols)
#df = df.drop(["Date", "Holiday", "Seasons"], axis=1)
df.head()

bike_count         hour  \
Date       Rented Bike Count Hour  Temperature(°C)  Humidity(%)   
01/12/2017 254               0                -5.2           37   
           204               1                -5.5           38   
           173               2                  -6           39   
           107               3                -6.2           40   

                                               temp          humidity  \
Date       Rented Bike Count Hour  Wind speed (m/s)  Visibility (10m)   
01/12/2017 254               0                  2.2              2000   
           204               1                  0.8              2000   
           173               2                    1              2000   
           107               3                  0.9              2000   

                                                        wind  \
Date       Rented Bike Count Hour  Dew point temperature(°C)   
01/12/2017 254               0                         -17.6   
           204               1                         -17.6   
           173               2                         -17.7   
           107               3                         -17.6   

                                                visibility   dew_pt_temp  \
Date       Rented Bike Count Hour  Solar Radiation (MJ/m2)  Rainfall(mm)   
01/12/2017 254               0                           0             0   
           204               1                           0             0   
           173               2                           0             0   
           107               3                           0             0   

                                       radiation     rain        snow  \
Date       Rented Bike Count Hour  Snowfall (cm)  Seasons     Holiday   
01/12/2017 254               0                 0   Winter  No Holiday   
           204               1                 0   Winter  No Holiday   
           173               2                 0   Winter  No Holiday   
           107               3                 0   Winter  No Holiday   

                                        functional  
Date       Rented Bike Count Hour  Functioning Day  
01/12/2017 254               0                 Yes  
           204               1                 Yes  
           173               2                 Yes  
           107               3                 Yes